In [0]:
# needed to import excel files
%pip install -i https://pypi.org/simple openpyxl

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
from datetime import datetime, timedelta 
from getpass import getuser as get_user 
from getpass import getpass as get_password 
from os import environ as enviroment_vars 
from os import setpgrp as set_process_group 
from random import random 
from shlex import split as cmd_split 
from string import punctuation 
from subprocess import Popen, PIPE, STDOUT 
import pandas as pd 

In [0]:
from pyspark import SparkContext, HiveContext, SparkConf  
from pyspark.sql import SparkSession 

ss = SparkSession(sc) 
ss 

In [0]:
#Calculate CYTD, PYTD Error Transmission based on application message code 003 in FIRST TRANSMISSION
#Data source: Data Lake
#Application Message Dim table is not in Data Lake; Read from the copy in the same folder
from datetime import datetime

vwdw_application_message_dim = pd.read_excel('vwdw_application_message_dim.xlsx').astype(str)
df1 = ss.createDataFrame(vwdw_application_message_dim)
df1.registerTempTable("vwdw_application_message_dim")

sql = """
select distinct
AMT.CMHC_LOAN_ACCOUNT_NBR,
SIT.LENDER_EN, 
SIT.LENDER_CDE,
SIT.PARENT_LENDER_EN, 
SIT.PARENT_LENDER_CODE,
amd.application_message_desc_en,
amd.application_message_desc_fr,
1 as error_cnt,
case 
when row_number() over (partition by AMT.CMHC_LOAN_ACCOUNT_NBR order by AMT.CMHC_LOAN_ACCOUNT_NBR) = 1
then 1 else 0
end as unique_err_loan

from cmwdw_insurance.tbdw_tgt_application_message_transaction_fact AMT

inner join vwdw_application_message_dim amd 
on (amt.application_message_surrkey = INT(amd.application_message_surrkey)) and amd.application_message_tcd="3"

LEFT join cmwdw_insurance.vwdw_loan_transmission_dim ltd 
on (amt.loan_transmission_Surrkey = ltd.loan_transmission_surrkey and ltd.transmission_first_ind=1) 

LEFT join sb_rmo_ho_reporting.lender_dashboard_tat_desj SIT 
on (ltd.cmhc_loan_account_nbr = sit.cmhc_loan_account_nbr)

/**where sit.REC_YEAR in ("CYTD", "PYTD")**/
where amt.application_identifier_ts>='2020-01-01 00:00:12.591090'
group by AMT.CMHC_LOAN_ACCOUNT_NBR,
SIT.LENDER_GROUP,
SIT.LENDER_EN, 
SIT.LENDER_CDE,
SIT.PARENT_LENDER_EN, 
SIT.PARENT_LENDER_CODE,
amd.application_message_desc_en,
amd.application_message_desc_fr
order by AMT.CMHC_LOAN_ACCOUNT_NBR
"""
err_df = ss.sql(sql)
err_df.write.mode("overwrite").format("orc").saveAsTable("sb_rmo_ho_reporting.lender_dashboard_error")
print("Errors First Transmission table row count: ", err_df.count())
print("Table creation for ERROR AT First Transmission completed at: ", str(datetime.now()))

/databricks/spark/python/pyspark/sql/dataframe.py:334: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


Errors First Transmission table row count:  1769763
Table creation for ERROR AT First Transmission completed at:  2024-09-06 15:49:22.287613
